# Build Climatology Tutorial

As a baseline method, we build a climatology for FMC based on the historical average at a particular hour of the day. This method is inspired by Schreck et all 2023. This notebook utilizes retrieval and filtering of 10-h dead FMC data from a RAWS stash maintained by Angel Farguell, demonstrated in notebook `ingest_fm10_stash_tutorial`.

Main processes:
- `build_climatology`: this function retrieves RAWS data from a stash given input time period and spatial domain. The data is saved to a local directory for potential reuse, such as repeated applications of calculating forecast error for cross validation. Parameters for this process are stored in `etc/params_models.yaml`
- `get_climatology_forecasts`: this function returns FMC forecasts

## Setup

In [ ]:
import os.path as osp
from dateutil.relativedelta import relativedelta
import json
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
sys.path.append('../src')
from utils import Dict, read_yml, read_pkl, str2time, print_dict_summary, time_range
# import ingest.retrieve_raws_api as rr
# import ingest.retrieve_raws_stash as rrs
import models.moisture_models as mm

In [ ]:
with open("../etc/training_data_config.json", "r") as json_file:
    config = json.load(json_file)   
    config = Dict(config)

In [ ]:
print_dict_summary(config)

## Climatology

Method description...

6 years


In [ ]:
start = str2time(config.start_time)
# end = start + relativedelta(hours=48)
end = start + relativedelta(hours = 12)

In [ ]:
import importlib
import models.moisture_models
importlib.reload(models.moisture_models)
import models.moisture_models as mm

In [ ]:
clim_dict = mm.build_climatology(
    start,
    end,
    config.bbox
)

In [ ]:
clim_df = mm.get_climatology_forecasts(clim_dict)

In [ ]:
clim_df

In [ ]:
# Plot a few timeseries

df_subset = clim_df.head(5).T  
plt.figure(figsize=(10, 6))
for idx in df_subset.columns:
    plt.plot(df_subset.index, df_subset[idx], label=str(idx))

plt.xlabel('Time')
plt.ylabel('Forecasted FMC (%)')
plt.title('Forecasted FMC over Time')
plt.legend(title='STID')
plt.grid(True)
plt.show()